In [ ]:
!pip install --quiet unstructured langchain langchain_experimental openai


In [ ]:
# Импортирование библиотек
import io
import os
from langchain.embeddings import Embeddings
from langchain_experimental.text_splitter import SemanticChunker
from unstructured.partition.api import partition_via_api
from unstructured.documents.elements import Element
from openai import OpenAI
from IPython.display import display, HTML

class CustomOllamaEmbeddings(Embeddings):
    def __init__(self):
        self.model_name = "bge-m3"
        self.client = OpenAI(
            base_url="http://10.41.56.9:12121/v1",  # ваш локальный OpenAI API
            api_key=os.environ.get("LLM_API_KEY", ""),  # ключ если нужен
        )

    def embed_documents(self, texts):
        return self._get_embeddings(texts)

    def embed_query(self, text):
        return self._get_embeddings([text])[0]

    def _get_embeddings(self, input_texts):
        response = self.client.embeddings.create(
            model=self.model_name,
            input=input_texts
        )
        return [e.embedding for e in response.data]

# Функция для загрузки файла
def load_file(file_path):
    with open(file_path, "rb") as f:
        return f.read()

# Функция для парсинга с использованием unstructured
def parse_with_unstructured(file_bytes, metadata_filename="document.docx"):
    file_obj = io.BytesIO(file_bytes)
    elements = partition_via_api(
        file=file_obj,
        api_url="http://localhost:8111/general/v0/general",
        metadata_filename=metadata_filename,
        strategy="auto",
        chunking_strategy="by_title",
        max_characters=1500,
        new_after_n_chars=1200,
        combine_under_n_chars=200,
    )
    return elements

# Функция для парсинга с использованием langchain
def parse_with_langchain(file_bytes):
    embeddings = CustomOllamaEmbeddings()
    splitter = SemanticChunker(embeddings)
    text = file_bytes.decode("utf-8")
    chunks = splitter.split_text(text)
    return chunks

# Загрузка файла
file_path = "path_to_your_file.docx"  # Укажите путь к вашему файлу
file_bytes = load_file(file_path)

# Парсинг с использованием unstructured
unstructured_elements = parse_with_unstructured(file_bytes)

# Парсинг с использованием langchain
langchain_chunks = parse_with_langchain(file_bytes)

# Отображение результатов
def display_chunks(elements, method_name):
    display(HTML(f"<h3>Результаты метода: {method_name}</h3>"))
    for i, el in enumerate(elements):
        text = el.text.strip() if hasattr(el, 'text') else el
        display(HTML(f"<b>Чанк {i + 1}:</b><br>{text[:500]}...<br><br>"))

# Отображение результатов обоих методов
display_chunks(unstructured_elements, "Unstructured")
display_chunks(langchain_chunks, "Langchain Semantic Chunking")